<h1><center> Python project proposal </center></h1>
<h2><center>Current car prices from bazos.cz</center></h2>
<h3><center>Daniel Brosko, Vojtěch Suchánek</center></h3>

Our goal is to web-scrape advertisements listed on website bazos.cz, which is currently one of the most used websites for selling used cars in Czech republic. It has more than 15 000 car adds daily. On the other hand, it has really poor search options, which pretty much complicates searching for desired car based on your parameters.

We are going to code algorithm, which will scan adds for the current day, pick those, which fulfill our conditions on date and car type and save their links. Then we will go to each link and save the text of the add. Then we will try to analyze the text of the add to find our parameters. In the end, we will perform some visualizations of distribution of price and selected parameters and try to select some underprices offers.

This task might be better to perform continuouslly, for example each hour, to not overload the website. This approach might also allow for longer time period to analyze the data. But since this project should be designed as one-time run, we decided to limit the data to only current date.

In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
bots = requests.get('https://auto.bazos.cz/robots.txt')
print(bots.text)

User-agent: *
Disallow: /*humkreis
Disallow: /*order=
Disallow: /*cenaod=
Disallow: /*vkm=
Disallow: /*gal=
Disallow: /suggest.php
Disallow: /suggestpsc.php
Disallow: /suggestpscinsert.php
Disallow: /overeni.php
Disallow: /detailtel.php
Disallow: /agent.php
Disallow: /favourite.php
Disallow: /kmailn.php
Disallow: /upload.php
Disallow: /zip.php
Disallow: /smazat/

User-agent: Mediapartners-Google
Disallow:

User-agent: trovitBot
Disallow: /

User-agent: AhrefsBot
Disallow: /

User-agent: pricebot
Disallow: /

User-agent: magpie-crawler
Disallow: /

User-agent: SemrushBot
Disallow: /

User-Agent: trendictionbot
Disallow: /

User-agent: SemrushBot
Disallow: /

User-Agent: trendictionbot
Disallow: /

User-agent: BLEXBot
Disallow: /

User-agent: Riddler
Disallow: /

Sitemap: https://auto.bazos.cz/sitemap.php


From the robots page we can see that our actions done in our projects are allowed, since we are not gonna use these search commands.

Here we will get the main page from car section of bazos.

In [4]:
number_sequence = range(20, 10000, 20) #start, stop (not included), step
# we create empty list for saving the urls,
# and then append other tabs with offers (since there are only 20 offers per tab by default)
main_url_list = list()
main_url_list.append('https://auto.bazos.cz/')

for i in range(0,len(number_sequence)):
    url = f'https://auto.bazos.cz/{number_sequence[i]}/'
    main_url_list.append(url)
# here we can check the list of urls for particular tabs
print(main_url_list)

['https://auto.bazos.cz/', 'https://auto.bazos.cz/20/', 'https://auto.bazos.cz/40/', 'https://auto.bazos.cz/60/', 'https://auto.bazos.cz/80/', 'https://auto.bazos.cz/100/', 'https://auto.bazos.cz/120/', 'https://auto.bazos.cz/140/', 'https://auto.bazos.cz/160/', 'https://auto.bazos.cz/180/', 'https://auto.bazos.cz/200/', 'https://auto.bazos.cz/220/', 'https://auto.bazos.cz/240/', 'https://auto.bazos.cz/260/', 'https://auto.bazos.cz/280/', 'https://auto.bazos.cz/300/', 'https://auto.bazos.cz/320/', 'https://auto.bazos.cz/340/', 'https://auto.bazos.cz/360/', 'https://auto.bazos.cz/380/', 'https://auto.bazos.cz/400/', 'https://auto.bazos.cz/420/', 'https://auto.bazos.cz/440/', 'https://auto.bazos.cz/460/', 'https://auto.bazos.cz/480/', 'https://auto.bazos.cz/500/', 'https://auto.bazos.cz/520/', 'https://auto.bazos.cz/540/', 'https://auto.bazos.cz/560/', 'https://auto.bazos.cz/580/', 'https://auto.bazos.cz/600/', 'https://auto.bazos.cz/620/', 'https://auto.bazos.cz/640/', 'https://auto.baz

In [5]:
# in the next step, we get the text of each of those tabs using the BeautifulSoup function,
# and save it as elements of the "soup_list"
soup_list = list()
for url in main_url_list:
    page = requests.get(url)
    soup_list.append(BeautifulSoup(page.text, 'html'))


In [7]:
# too long to print
#print(soup_list)

There are 20 adds plus other page elements. Lets pick just one add to see their construction.

There are several elements, which are important to us. In this section, we are interested in add id, which we are able to extract from href in element a on the second line. Its the number after "inzerat/". We are also interested in the href itself, since it is our link to follow to the actual add. Another element important to us is h2 class = "nadpis", which contains title of the add, from which we can extract type of car. And the last interesting element is span with class "velikost10", which reffers to date uploaded.

Now we can follow the link to the actual add page to see its content.

In [8]:
# by the following code, we get the urls of each advertisement /offer/ (listed in the tabs we work with),
# and save it to "list_of_offers_url"
list_of_offers_url = list()
for element in soup_list:
    x = element.find('div', {'class':'inzeraty inzeratyflex'})
    y = x.find('div', {'class':'inzeratynadpis'})
    z = y.find('a')
    w = z.get('href')
    list_of_offers_url.append(f'https://auto.bazos.cz{w}')

print(list_of_offers_url)

['https://auto.bazos.cz/inzerat/156899528/seat-leon-st-16tdi-85kw-dsg-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/156889523/qashqai-15-dci-rok-2011-122000-km-81-kw.php', 'https://auto.bazos.cz/inzerat/156852921/skoda-kodiaq-20tdi-110kw-dsg-lk-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/156800462/skoda-octavia-3-16tdi-85kw-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/156701370/hyundai-i30-14-i-benzin-6-rychlosti-puvod-cr-od-1majitele.php', 'https://auto.bazos.cz/inzerat/154947423/skoda-octavia-16-tdi-81-kw-combi-style-od-1maj-puvod-cr.php', 'https://auto.bazos.cz/inzerat/156532359/skoda-octavia-14-tsi-cng.php', 'https://auto.bazos.cz/inzerat/156590680/corolla.php', 'https://auto.bazos.cz/inzerat/156457126/vw-passat-b8-20tdi-110kw-dsg-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/156890473/skoda-superb-elegance-25tdi-manual-nova-stkbixenontop.php', 'https://auto.bazos.cz/inzerat/156352222/vw-passat-20-tdi-103-kw-com

In [ ]:
#similar as above for class = 'velikos10', 'nadpis', ...
# also the code below will be automatized using for loops, OOP, and etc.

In [ ]:
# by the following code, we get the source text of each advertisement, again using the BeautifulSoup function,
# then we save that into a list "list_of_source_text_of_adv"
list_of_source_text_of_adv = list()
for url_element in list_of_offers_url:
    add_page = requests.get(url_element)
    soup_add = BeautifulSoup(add_page.text, 'html')
    list_of_offers_url.append(soup_add)

In [14]:
# we will print just first element to check the content, since doing that for entire list would take too much time,
# and probably even the output  would be too large to print
print(list_of_source_text_of_adv)

[]


In [9]:
#continue here??? - maybe start here the text mining part? - if the code above works,
# I (Daniel) will make it as OOP using class
add_page = requests.get('https://auto.bazos.cz/inzerat/151109412/volkswagen-golf-gtd-135kw-manual-vyhrev-navi-led.php')
soup_add = BeautifulSoup(add_page.text, 'html')
print(soup_add.find('div', {'class':'popisdetail'}))

None


Here we can see the description, which we will use to extract parameters of the car. This will be one of the most difficult parts of the project to extract the relevant text in unformated text. There is no official format to the text, so we will have to find a way how to extract this information from various formats.

After retrieving these informations, we will use analysis of the data got from this process.